In [43]:
#dependencies:
import pandas as pd

import torch
from transformers import GPT2Tokenizer

from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer

In [44]:
#loading datasets:
from datasets import load_dataset

ds = load_dataset("stanfordnlp/SHP", split='train')

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [45]:
df = ds.to_pandas()
print(df.columns)


Index(['post_id', 'domain', 'upvote_ratio', 'history', 'c_root_id_A',
       'c_root_id_B', 'created_at_utc_A', 'created_at_utc_B', 'score_A',
       'score_B', 'human_ref_A', 'human_ref_B', 'labels', 'seconds_difference',
       'score_ratio'],
      dtype='object')


In [46]:
# df['response_length'] = df['history'].apply(len)
# df['label'] = df['response_length'].apply(lambda x: 'long' if x > 100 else 'short')
df.drop(columns=['post_id', 'domain', 'c_root_id_A', 'c_root_id_B', 'created_at_utc_A', 'created_at_utc_B', 'seconds_difference'])

,upvote_ratio,history,score_A,score_B,human_ref_A,human_ref_B,labels,score_ratio
0,0.99,In an interview right before receiving the 201...,52,54,Currently wrapping up my PhD. There is a stark...,It’s ironic to me that research has shown that...,0,1.038462
1,0.95,If any professor is reading this: please do no...,5,17,And when your teacher doesn't listen or pay at...,"I'm pretty strict on time, to the point where ...",0,3.400000
2,0.95,If any professor is reading this: please do no...,5,7,Profs can be oblivious? What’s new!,This sounds like a problem with a specific pro...,0,1.400000
3,0.95,If any professor is reading this: please do no...,7,5,This sounds like a problem with a specific pro...,And when your teacher doesn't listen or pay at...,1,1.400000
4,0.95,If any professor is reading this: please do no...,6,7,This would be totally unacceptable in my class...,This sounds like a problem with a specific pro...,0,1.166667
...,...,...,...,...,...,...,...,...
348713,0.94,Can I get in trouble for giving my neighbor hi...,7,25,Just put up a fence. Legally he isn't responsi...,"Whatever you do, don't cut his trees down.",0,3.571429
348714,0.94,Can I get in trouble for giving my neighbor hi...,2,25,"If OP pays someone to clean his yard, and then...","Whatever you do, don't cut his trees down.",0,12.500000
348715,0.94,Can I get in trouble for giving my neighbor hi...,9,7,My observation is that both of you are idiots...,Are you Rand Paul's neighbor? https://www.gq....,1,1.285714
348716,0.94,Can I get in trouble for giving my neighbor hi...,9,7,My observation is that both of you are idiots...,Just put up a fence. Legally he isn't responsi...,1,1.285714


In [47]:
model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

/Users/riddhib/.pyenv/versions/3.10.13/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [48]:
from trl_rlhf_data import runner, ScriptArguments
import re
from dataclasses import dataclass
from typing import Dict, List, Optional

from datasets import load_dataset
from transformers import HfArgumentParser

In [49]:
dataset = runner(ScriptArguments)

TypeError: runner() takes 0 positional arguments but 1 was given